In [ ]:

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("BibleCounter").getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('WARN')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/27 21:42:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


25/02/27 21:42:58 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 51317)
Traceback (most recent call last):
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/socketserver.py", line 360, in fini

In [ ]:
# Download the bible as txt
!curl -L "https://www.gutenberg.org/cache/epub/10/pg10.txt" > bible.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4351k  100 4351k    0     0  5498k      0 --:--:-- --:--:-- --:--:-- 5494k


25/02/27 00:18:35 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1242955 ms exceeds timeout 120000 ms
25/02/27 00:18:35 WARN SparkContext: Killing executors is not supported by current scheduler.
25/02/27 01:52:37 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$

In [2]:
# Use SparkSQL to list all the words in the bible with their counts sorted descending by count.
# Turn in your code and sample of your results (at least top 20). 
# Be sure to ignore case and punctuation and eliminate the blank lines.

lines = sc.textFile("bible.txt")


In [3]:
import re
words = lines.flatMap(lambda line: re.findall(r'\b\w+\b', line.lower()))


In [5]:
wordCounts = words.filter(lambda word: word != "") \
                   .map(lambda word: (word, 1)) \
                   .reduceByKey(lambda a, b: a + b)
                   
sortedWordCounts = wordCounts.sortBy(lambda pair: pair[1], ascending=False)
top10 = sortedWordCounts.take(10)
for word, count in top10:
    print(word, count)

the 64309
and 51762
of 34846
to 13680
that 12927
in 12727
he 10422
shall 9840
for 8997
unto 8997


In [6]:
wordCountList = wordCounts.collect()
wordCountDF = spark.createDataFrame(wordCounts, ["word", "count"])
wordCountDF.registerTempTable("wordCount")

/Users/mckaysnell/BYU-CS/CS-452/SparkSQL/venv/lib/python3.9/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [7]:
wordCountDF.show()

+------------+-----+
|        word|count|
+------------+-----+
|     project|   88|
|   gutenberg|   97|
|       ebook|   13|
|          of|34846|
|        king| 2548|
|       james|   52|
|     version|    7|
|       bible|    8|
|        this| 2830|
|          is| 7012|
|         use|   49|
|      anyone|    5|
|    anywhere|    2|
|          in|12727|
|      united|   16|
|       other|  481|
|       world|  290|
|          at| 1583|
|          no| 1404|
|restrictions|    2|
+------------+-----+
only showing top 20 rows



In [8]:
top10DF = spark.sql("""
    SELECT word, count 
    FROM wordCount 
    ORDER BY count DESC 
    LIMIT 10
""")
top10DF.show()

+-----+-----+
| word|count|
+-----+-----+
|  the|64309|
|  and|51762|
|   of|34846|
|   to|13680|
| that|12927|
|   in|12727|
|   he|10422|
|shall| 9840|
|  for| 8997|
| unto| 8997|
+-----+-----+



25/02/28 01:30:56 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 295712 ms exceeds timeout 120000 ms
25/02/28 01:30:56 WARN SparkContext: Killing executors is not supported by current scheduler.
25/02/28 01:31:03 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [ ]:
# What to try something else? 